# 📊 Datathon FIAP - Passos Mágicos
## Respostas às 11 Perguntas de Negócio

Este notebook apresenta análises detalhadas para responder às 11 perguntas propostas no Datathon, utilizando os dados do PEDE (Pesquisa Extensiva do Desenvolvimento Educacional) dos anos 2022, 2023 e 2024.

**Autor:** Leandro Leme Crespo

---

## 1. Configuração do Ambiente

In [ ]:
# Clonar o repositório do GitHub (executar apenas no Google Colab)
!git clone https://github.com/LeandroCrespo/datathon-passos-magicos.git
print('✅ Repositório clonado com sucesso!')

In [ ]:
# Instalar bibliotecas necessárias
!pip install openpyxl plotly kaleido scipy -q

In [ ]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import warnings

warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Cores personalizadas
CORES_PEDRAS = {
    'Quartzo': '#C0C0C0',
    'Ágata': '#9370DB',
    'Agata': '#9370DB',
    'Ametista': '#8B008B',
    'Topázio': '#FFD700'
}

CORES_ANOS = {2022: '#3498db', 2023: '#e74c3c', 2024: '#2ecc71'}

print('✅ Bibliotecas importadas com sucesso!')

In [ ]:
# Carregar dados (do repositório GitHub)
CAMINHO_ARQUIVO = '/content/datathon-passos-magicos/data/BASE_DE_DADOS_PEDE_2024_DATATHON.xlsx'

xlsx = pd.ExcelFile(CAMINHO_ARQUIVO)
df_2022 = pd.read_excel(xlsx, sheet_name='PEDE2022')
df_2023 = pd.read_excel(xlsx, sheet_name='PEDE2023')
df_2024 = pd.read_excel(xlsx, sheet_name='PEDE2024')

print(f'📊 Dados carregados:')
print(f'   PEDE 2022: {df_2022.shape[0]:,} alunos')
print(f'   PEDE 2023: {df_2023.shape[0]:,} alunos')
print(f'   PEDE 2024: {df_2024.shape[0]:,} alunos')

In [ ]:
# Preparar dados unificados
def preparar_dados(df, ano):
    df_prep = df.copy()
    df_prep['ANO'] = ano
    
    col_map = {}
    for col in df_prep.columns:
        col_lower = col.lower()
        if col_lower == 'iaa': col_map[col] = 'IAA'
        elif col_lower == 'ieg' and 'destaque' not in col_lower: col_map[col] = 'IEG'
        elif col_lower == 'ips': col_map[col] = 'IPS'
        elif col_lower == 'ipp': col_map[col] = 'IPP'
        elif col_lower == 'ida' and 'destaque' not in col_lower: col_map[col] = 'IDA'
        elif col_lower == 'ipv' and 'destaque' not in col_lower: col_map[col] = 'IPV'
        elif col_lower == 'ian': col_map[col] = 'IAN'
        elif 'defas' in col_lower: col_map[col] = 'DEFASAGEM'
        elif col_lower == 'fase': col_map[col] = 'FASE'
        elif col_lower == 'gênero' or col_lower == 'genero': col_map[col] = 'GENERO'
        elif 'atingiu pv' in col_lower: col_map[col] = 'PONTO_VIRADA'
    
    if ano == 2022:
        col_map['INDE 22'] = 'INDE'
        col_map['Pedra 22'] = 'PEDRA'
    elif ano == 2023:
        col_map['INDE 2023'] = 'INDE'
        col_map['Pedra 2023'] = 'PEDRA'
    elif ano == 2024:
        col_map['INDE 2024'] = 'INDE'
        col_map['Pedra 2024'] = 'PEDRA'
    
    df_prep = df_prep.rename(columns=col_map)
    return df_prep

df_2022_prep = preparar_dados(df_2022, 2022)
df_2023_prep = preparar_dados(df_2023, 2023)
df_2024_prep = preparar_dados(df_2024, 2024)

# Unificar dados
colunas = ['ANO', 'INDE', 'IAA', 'IEG', 'IPS', 'IPP', 'IDA', 'IPV', 'IAN', 'DEFASAGEM', 'PEDRA', 'GENERO', 'PONTO_VIRADA', 'FASE']

df_unificado = pd.concat([
    df_2022_prep[[c for c in colunas if c in df_2022_prep.columns]],
    df_2023_prep[[c for c in colunas if c in df_2023_prep.columns]],
    df_2024_prep[[c for c in colunas if c in df_2024_prep.columns]]
], ignore_index=True)

# Converter colunas numéricas
for col in ['INDE', 'IAA', 'IEG', 'IPS', 'IPP', 'IDA', 'IPV', 'IAN', 'DEFASAGEM']:
    if col in df_unificado.columns:
        df_unificado[col] = pd.to_numeric(df_unificado[col], errors='coerce')

# Padronizar PEDRA
df_unificado['PEDRA'] = df_unificado['PEDRA'].replace({'Agata': 'Ágata'})

print(f'\n✅ DataFrame unificado: {len(df_unificado):,} registros')

---

## 📋 PERGUNTA 1: Adequação do Nível (IAN)

**Qual é o perfil geral de defasagem dos alunos (IAN) e como ele evolui ao longo do ano?**

*Exemplo: quantos alunos estão moderadamente ou severamente defasados?*

In [ ]:
# Classificar defasagem em categorias
def classificar_defasagem(valor):
    if pd.isna(valor):
        return 'Não informado'
    elif valor <= -3:
        return 'Severamente defasado (≤-3)'
    elif valor <= -1:
        return 'Moderadamente defasado (-2 a -1)'
    elif valor == 0:
        return 'Adequado ao nível (0)'
    else:
        return 'Acima do nível (≥1)'

df_unificado['CATEGORIA_DEFASAGEM'] = df_unificado['DEFASAGEM'].apply(classificar_defasagem)

# Análise por ano
print('='*70)
print('PERGUNTA 1: PERFIL DE DEFASAGEM DOS ALUNOS (IAN)')
print('='*70)

for ano in [2022, 2023, 2024]:
    df_ano = df_unificado[df_unificado['ANO'] == ano]
    total = len(df_ano)
    
    print(f'\n📅 ANO {ano} (n={total:,})')
    print('-'*40)
    
    # Estatísticas da defasagem
    defasagem = df_ano['DEFASAGEM'].dropna()
    print(f'Média de defasagem: {defasagem.mean():.2f} anos')
    print(f'Mediana: {defasagem.median():.1f} anos')
    print(f'Desvio padrão: {defasagem.std():.2f}')
    
    # Distribuição por categoria
    print(f'\nDistribuição por categoria:')
    cat_counts = df_ano['CATEGORIA_DEFASAGEM'].value_counts()
    for cat, count in cat_counts.items():
        pct = count / total * 100
        print(f'  {cat}: {count:,} ({pct:.1f}%)')

In [ ]:
# Visualização da evolução da defasagem
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Gráfico 1: Distribuição por categoria e ano
ordem_cat = ['Severamente defasado (≤-3)', 'Moderadamente defasado (-2 a -1)', 
             'Adequado ao nível (0)', 'Acima do nível (≥1)']
cores_cat = ['#d62728', '#ff7f0e', '#2ca02c', '#1f77b4']

cat_ano = df_unificado.groupby(['ANO', 'CATEGORIA_DEFASAGEM']).size().unstack(fill_value=0)
cat_ano_pct = cat_ano.div(cat_ano.sum(axis=1), axis=0) * 100
cat_ano_pct = cat_ano_pct[[c for c in ordem_cat if c in cat_ano_pct.columns]]

cat_ano_pct.plot(kind='bar', stacked=True, ax=axes[0], color=cores_cat[:len(cat_ano_pct.columns)])
axes[0].set_title('Distribuição da Defasagem por Ano', fontweight='bold')
axes[0].set_xlabel('Ano')
axes[0].set_ylabel('Percentual (%)')
axes[0].legend(title='Categoria', bbox_to_anchor=(1.02, 1), fontsize=8)
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)

# Gráfico 2: Evolução da média de defasagem
media_def = df_unificado.groupby('ANO')['DEFASAGEM'].mean()
axes[1].plot(media_def.index, media_def.values, marker='o', linewidth=3, markersize=12, color='#2E86AB')
axes[1].fill_between(media_def.index, media_def.values, alpha=0.3, color='#2E86AB')
axes[1].axhline(y=0, color='green', linestyle='--', alpha=0.7, label='Nível adequado')
axes[1].set_title('Evolução da Média de Defasagem', fontweight='bold')
axes[1].set_xlabel('Ano')
axes[1].set_ylabel('Média de Defasagem (anos)')
for x, y in zip(media_def.index, media_def.values):
    axes[1].annotate(f'{y:.2f}', (x, y), textcoords='offset points', xytext=(0, 10), ha='center', fontweight='bold')
axes[1].legend()

# Gráfico 3: Evolução do IAN médio
media_ian = df_unificado.groupby('ANO')['IAN'].mean()
axes[2].plot(media_ian.index, media_ian.values, marker='s', linewidth=3, markersize=12, color='#E74C3C')
axes[2].fill_between(media_ian.index, media_ian.values, alpha=0.3, color='#E74C3C')
axes[2].set_title('Evolução do IAN Médio', fontweight='bold')
axes[2].set_xlabel('Ano')
axes[2].set_ylabel('IAN Médio')
for x, y in zip(media_ian.index, media_ian.values):
    axes[2].annotate(f'{y:.2f}', (x, y), textcoords='offset points', xytext=(0, 10), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('pergunta1_defasagem.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n📊 CONCLUSÃO PERGUNTA 1:')
print('A defasagem média dos alunos está MELHORANDO ao longo dos anos:')
print(f'  • 2022: {media_def[2022]:.2f} anos de defasagem média')
print(f'  • 2023: {media_def[2023]:.2f} anos de defasagem média')
print(f'  • 2024: {media_def[2024]:.2f} anos de defasagem média')
print(f'\nMelhoria de {abs(media_def[2024] - media_def[2022]):.2f} anos entre 2022 e 2024.')

---

## 📋 PERGUNTA 2: Desempenho Acadêmico (IDA)

**O desempenho acadêmico médio (IDA) está melhorando, estagnado ou caindo ao longo das fases e anos?**

In [ ]:
print('='*70)
print('PERGUNTA 2: EVOLUÇÃO DO DESEMPENHO ACADÊMICO (IDA)')
print('='*70)

# Estatísticas do IDA por ano
ida_stats = df_unificado.groupby('ANO')['IDA'].agg(['mean', 'median', 'std', 'count'])
print('\n📊 Estatísticas do IDA por Ano:')
print(ida_stats.round(3))

# Teste estatístico de tendência
ida_2022 = df_unificado[df_unificado['ANO'] == 2022]['IDA'].dropna()
ida_2023 = df_unificado[df_unificado['ANO'] == 2023]['IDA'].dropna()
ida_2024 = df_unificado[df_unificado['ANO'] == 2024]['IDA'].dropna()

# ANOVA para comparar os anos
f_stat, p_value = stats.f_oneway(ida_2022, ida_2023, ida_2024)
print(f'\n📈 Teste ANOVA (comparação entre anos):')
print(f'   F-statistic: {f_stat:.3f}')
print(f'   P-value: {p_value:.6f}')
if p_value < 0.05:
    print('   ⚠️ Há diferença estatisticamente significativa entre os anos (p < 0.05)')
else:
    print('   ✅ Não há diferença estatisticamente significativa entre os anos')

In [ ]:
# Visualização do IDA
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Gráfico 1: Boxplot por ano
df_unificado.boxplot(column='IDA', by='ANO', ax=axes[0])
axes[0].set_title('Distribuição do IDA por Ano', fontweight='bold')
axes[0].set_xlabel('Ano')
axes[0].set_ylabel('IDA')
plt.suptitle('')

# Gráfico 2: Evolução da média
media_ida = df_unificado.groupby('ANO')['IDA'].mean()
axes[1].plot(media_ida.index, media_ida.values, marker='o', linewidth=3, markersize=12, color='#9b59b6')
axes[1].fill_between(media_ida.index, media_ida.values, alpha=0.3, color='#9b59b6')
axes[1].set_title('Evolução do IDA Médio', fontweight='bold')
axes[1].set_xlabel('Ano')
axes[1].set_ylabel('IDA Médio')
for x, y in zip(media_ida.index, media_ida.values):
    axes[1].annotate(f'{y:.2f}', (x, y), textcoords='offset points', xytext=(0, 10), ha='center', fontweight='bold')

# Gráfico 3: IDA por Pedra
ordem_pedras = ['Quartzo', 'Ágata', 'Ametista', 'Topázio']
ida_pedra = df_unificado.groupby('PEDRA')['IDA'].mean().reindex([p for p in ordem_pedras if p in df_unificado['PEDRA'].unique()])
cores = [CORES_PEDRAS.get(p, '#808080') for p in ida_pedra.index]
bars = axes[2].bar(ida_pedra.index, ida_pedra.values, color=cores, edgecolor='black')
axes[2].set_title('IDA Médio por Pedra', fontweight='bold')
axes[2].set_xlabel('Pedra')
axes[2].set_ylabel('IDA Médio')
for bar, val in zip(bars, ida_pedra.values):
    axes[2].annotate(f'{val:.2f}', (bar.get_x() + bar.get_width()/2, bar.get_height()), 
                     textcoords='offset points', xytext=(0, 5), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('pergunta2_ida.png', dpi=150, bbox_inches='tight')
plt.show()

# Conclusão
tendencia = 'ESTÁVEL' if abs(media_ida[2024] - media_ida[2022]) < 0.3 else ('MELHORANDO' if media_ida[2024] > media_ida[2022] else 'CAINDO')
print(f'\n📊 CONCLUSÃO PERGUNTA 2:')
print(f'O desempenho acadêmico (IDA) está {tendencia}:')
print(f'  • 2022: IDA médio = {media_ida[2022]:.2f}')
print(f'  • 2023: IDA médio = {media_ida[2023]:.2f}')
print(f'  • 2024: IDA médio = {media_ida[2024]:.2f}')
print(f'\nVariação: {((media_ida[2024] - media_ida[2022]) / media_ida[2022] * 100):.1f}% entre 2022 e 2024')

---

## 📋 PERGUNTA 3: Engajamento nas Atividades (IEG)

**O grau de engajamento dos alunos (IEG) tem relação direta com seus indicadores de desempenho (IDA) e do ponto de virada (IPV)?**

In [ ]:
print('='*70)
print('PERGUNTA 3: RELAÇÃO ENTRE ENGAJAMENTO (IEG), IDA E IPV')
print('='*70)

# Calcular correlações
corr_ieg_ida = df_unificado[['IEG', 'IDA']].corr().iloc[0, 1]
corr_ieg_ipv = df_unificado[['IEG', 'IPV']].corr().iloc[0, 1]
corr_ieg_inde = df_unificado[['IEG', 'INDE']].corr().iloc[0, 1]

print(f'\n📊 Correlações do IEG (Engajamento):')
print(f'   IEG x IDA (Desempenho): {corr_ieg_ida:.3f}')
print(f'   IEG x IPV (Ponto de Virada): {corr_ieg_ipv:.3f}')
print(f'   IEG x INDE (Índice Global): {corr_ieg_inde:.3f}')

# Interpretação
def interpretar_correlacao(r):
    if abs(r) < 0.3:
        return 'fraca'
    elif abs(r) < 0.7:
        return 'moderada'
    else:
        return 'forte'

print(f'\n📈 Interpretação:')
print(f'   • Correlação IEG-IDA: {interpretar_correlacao(corr_ieg_ida)} ({"positiva" if corr_ieg_ida > 0 else "negativa"})')
print(f'   • Correlação IEG-IPV: {interpretar_correlacao(corr_ieg_ipv)} ({"positiva" if corr_ieg_ipv > 0 else "negativa"})')

In [ ]:
# Visualização das relações
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Gráfico 1: IEG vs IDA (remover NaN de ambas as colunas juntas)
df_plot1 = df_unificado[['IEG', 'IDA']].dropna()
axes[0].scatter(df_plot1['IEG'], df_plot1['IDA'], alpha=0.3, c='#3498db')
z = np.polyfit(df_plot1['IEG'], df_plot1['IDA'], 1)
p = np.poly1d(z)
x_line = np.linspace(df_plot1['IEG'].min(), df_plot1['IEG'].max(), 100)
axes[0].plot(x_line, p(x_line), 'r--', linewidth=2, label=f'r = {corr_ieg_ida:.3f}')
axes[0].set_title('IEG vs IDA', fontweight='bold')
axes[0].set_xlabel('IEG (Engajamento)')
axes[0].set_ylabel('IDA (Desempenho Acadêmico)')
axes[0].legend()

# Gráfico 2: IEG vs IPV
df_plot2 = df_unificado[['IEG', 'IPV']].dropna()
axes[1].scatter(df_plot2['IEG'], df_plot2['IPV'], alpha=0.3, c='#e74c3c')
z2 = np.polyfit(df_plot2['IEG'], df_plot2['IPV'], 1)
p2 = np.poly1d(z2)
x_line2 = np.linspace(df_plot2['IEG'].min(), df_plot2['IEG'].max(), 100)
axes[1].plot(x_line2, p2(x_line2), 'b--', linewidth=2, label=f'r = {corr_ieg_ipv:.3f}')
axes[1].set_title('IEG vs IPV', fontweight='bold')
axes[1].set_xlabel('IEG (Engajamento)')
axes[1].set_ylabel('IPV (Ponto de Virada)')
axes[1].legend()

# Gráfico 3: IEG vs INDE
df_plot3 = df_unificado[['IEG', 'INDE']].dropna()
axes[2].scatter(df_plot3['IEG'], df_plot3['INDE'], alpha=0.3, c='#2ecc71')
z3 = np.polyfit(df_plot3['IEG'], df_plot3['INDE'], 1)
p3 = np.poly1d(z3)
x_line3 = np.linspace(df_plot3['IEG'].min(), df_plot3['IEG'].max(), 100)
axes[2].plot(x_line3, p3(x_line3), 'purple', linestyle='--', linewidth=2, label=f'r = {corr_ieg_inde:.3f}')
axes[2].set_title('IEG vs INDE', fontweight='bold')
axes[2].set_xlabel('IEG (Engajamento)')
axes[2].set_ylabel('INDE (Índice Global)')
axes[2].legend()

plt.tight_layout()
plt.savefig('pergunta3_engajamento.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n📊 CONCLUSÃO PERGUNTA 3:')
print(f'SIM, o engajamento (IEG) tem relação direta com o desempenho:')
print(f'  • Correlação moderada-forte com IDA (r={corr_ieg_ida:.3f})')
print(f'  • Correlação moderada-forte com IPV (r={corr_ieg_ipv:.3f})')
print(f'  • Correlação forte com INDE (r={corr_ieg_inde:.3f})')
print(f'\nAlunos mais engajados tendem a ter melhor desempenho acadêmico e maior probabilidade de atingir o ponto de virada.')

---

## 📋 PERGUNTA 4: Autoavaliação (IAA)

**As percepções dos alunos sobre si mesmos (IAA) são coerentes com seu desempenho real (IDA) e engajamento (IEG)?**

In [ ]:
print('='*70)
print('PERGUNTA 4: COERÊNCIA ENTRE AUTOAVALIAÇÃO (IAA) E DESEMPENHO REAL')
print('='*70)

# Correlações
corr_iaa_ida = df_unificado[['IAA', 'IDA']].corr().iloc[0, 1]
corr_iaa_ieg = df_unificado[['IAA', 'IEG']].corr().iloc[0, 1]
corr_iaa_inde = df_unificado[['IAA', 'INDE']].corr().iloc[0, 1]

print(f'\n📊 Correlações do IAA (Autoavaliação):')
print(f'   IAA x IDA (Desempenho): {corr_iaa_ida:.3f}')
print(f'   IAA x IEG (Engajamento): {corr_iaa_ieg:.3f}')
print(f'   IAA x INDE (Índice Global): {corr_iaa_inde:.3f}')

# Análise de discrepância
df_unificado['DISCREPANCIA_IAA_IDA'] = df_unificado['IAA'] - df_unificado['IDA']
media_discrepancia = df_unificado['DISCREPANCIA_IAA_IDA'].mean()
std_discrepancia = df_unificado['DISCREPANCIA_IAA_IDA'].std()

print(f'\n📈 Análise de Discrepância (IAA - IDA):')
print(f'   Média: {media_discrepancia:.2f}')
print(f'   Desvio Padrão: {std_discrepancia:.2f}')
if media_discrepancia > 0.5:
    print('   ⚠️ Alunos tendem a SUPERESTIMAR seu desempenho')
elif media_discrepancia < -0.5:
    print('   ⚠️ Alunos tendem a SUBESTIMAR seu desempenho')
else:
    print('   ✅ Autoavaliação relativamente alinhada com desempenho real')

In [ ]:
# Visualização
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Gráfico 1: IAA vs IDA
axes[0].scatter(df_unificado['IAA'], df_unificado['IDA'], alpha=0.3, c='#3498db')
axes[0].plot([0, 10], [0, 10], 'r--', linewidth=2, label='Linha de igualdade')
axes[0].set_title(f'IAA vs IDA (r={corr_iaa_ida:.3f})', fontweight='bold')
axes[0].set_xlabel('IAA (Autoavaliação)')
axes[0].set_ylabel('IDA (Desempenho Real)')
axes[0].legend()
axes[0].set_xlim(0, 10)
axes[0].set_ylim(0, 10)

# Gráfico 2: Distribuição da discrepância
axes[1].hist(df_unificado['DISCREPANCIA_IAA_IDA'].dropna(), bins=30, edgecolor='black', alpha=0.7, color='#9b59b6')
axes[1].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Sem discrepância')
axes[1].axvline(x=media_discrepancia, color='green', linestyle='-', linewidth=2, label=f'Média: {media_discrepancia:.2f}')
axes[1].set_title('Distribuição da Discrepância (IAA - IDA)', fontweight='bold')
axes[1].set_xlabel('Discrepância')
axes[1].set_ylabel('Frequência')
axes[1].legend()

# Gráfico 3: Discrepância por Pedra
disc_pedra = df_unificado.groupby('PEDRA')['DISCREPANCIA_IAA_IDA'].mean().reindex([p for p in ordem_pedras if p in df_unificado['PEDRA'].unique()])
cores = [CORES_PEDRAS.get(p, '#808080') for p in disc_pedra.index]
bars = axes[2].bar(disc_pedra.index, disc_pedra.values, color=cores, edgecolor='black')
axes[2].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[2].set_title('Discrepância Média por Pedra', fontweight='bold')
axes[2].set_xlabel('Pedra')
axes[2].set_ylabel('Discrepância Média (IAA - IDA)')
for bar, val in zip(bars, disc_pedra.values):
    axes[2].annotate(f'{val:.2f}', (bar.get_x() + bar.get_width()/2, bar.get_height()), 
                     textcoords='offset points', xytext=(0, 5), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('pergunta4_autoavaliacao.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n📊 CONCLUSÃO PERGUNTA 4:')
coerencia = 'PARCIALMENTE COERENTES' if abs(corr_iaa_ida) > 0.3 else 'POUCO COERENTES'
print(f'As autoavaliações são {coerencia} com o desempenho real:')
print(f'  • Correlação IAA-IDA: {corr_iaa_ida:.3f} ({interpretar_correlacao(corr_iaa_ida)})')
print(f'  • Discrepância média: {media_discrepancia:.2f}')
if media_discrepancia > 0:
    print(f'  • Tendência: alunos superestimam levemente seu desempenho')

---

## 📋 PERGUNTA 5: Aspectos Psicossociais (IPS)

**Há padrões psicossociais (IPS) que antecedem quedas de desempenho acadêmico ou de engajamento?**

In [ ]:
print('='*70)
print('PERGUNTA 5: PADRÕES PSICOSSOCIAIS (IPS) E DESEMPENHO')
print('='*70)

# Correlações do IPS
corr_ips_ida = df_unificado[['IPS', 'IDA']].corr().iloc[0, 1]
corr_ips_ieg = df_unificado[['IPS', 'IEG']].corr().iloc[0, 1]
corr_ips_inde = df_unificado[['IPS', 'INDE']].corr().iloc[0, 1]

print(f'\n📊 Correlações do IPS (Aspectos Psicossociais):')
print(f'   IPS x IDA (Desempenho): {corr_ips_ida:.3f}')
print(f'   IPS x IEG (Engajamento): {corr_ips_ieg:.3f}')
print(f'   IPS x INDE (Índice Global): {corr_ips_inde:.3f}')

# Análise por quartis de IPS
df_unificado['IPS_QUARTIL'] = pd.qcut(df_unificado['IPS'], q=4, labels=['Q1 (Baixo)', 'Q2', 'Q3', 'Q4 (Alto)'])

print(f'\n📈 Desempenho médio por quartil de IPS:')
ips_quartil_stats = df_unificado.groupby('IPS_QUARTIL')[['IDA', 'IEG', 'INDE']].mean()
print(ips_quartil_stats.round(2))

In [ ]:
# Visualização
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Gráfico 1: IPS vs IDA
axes[0].scatter(df_unificado['IPS'], df_unificado['IDA'], alpha=0.3, c='#e74c3c')
axes[0].set_title(f'IPS vs IDA (r={corr_ips_ida:.3f})', fontweight='bold')
axes[0].set_xlabel('IPS (Psicossocial)')
axes[0].set_ylabel('IDA (Desempenho)')

# Gráfico 2: Desempenho por quartil de IPS
x = np.arange(len(ips_quartil_stats))
width = 0.25
axes[1].bar(x - width, ips_quartil_stats['IDA'], width, label='IDA', color='#3498db')
axes[1].bar(x, ips_quartil_stats['IEG'], width, label='IEG', color='#2ecc71')
axes[1].bar(x + width, ips_quartil_stats['INDE'], width, label='INDE', color='#9b59b6')
axes[1].set_title('Indicadores por Quartil de IPS', fontweight='bold')
axes[1].set_xlabel('Quartil de IPS')
axes[1].set_ylabel('Média')
axes[1].set_xticks(x)
axes[1].set_xticklabels(ips_quartil_stats.index)
axes[1].legend()

# Gráfico 3: Evolução do IPS por ano
media_ips = df_unificado.groupby('ANO')['IPS'].mean()
axes[2].plot(media_ips.index, media_ips.values, marker='o', linewidth=3, markersize=12, color='#e74c3c')
axes[2].fill_between(media_ips.index, media_ips.values, alpha=0.3, color='#e74c3c')
axes[2].set_title('Evolução do IPS Médio', fontweight='bold')
axes[2].set_xlabel('Ano')
axes[2].set_ylabel('IPS Médio')
for x, y in zip(media_ips.index, media_ips.values):
    axes[2].annotate(f'{y:.2f}', (x, y), textcoords='offset points', xytext=(0, 10), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('pergunta5_psicossocial.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n📊 CONCLUSÃO PERGUNTA 5:')
print(f'O IPS tem correlação FRACA com desempenho acadêmico (r={corr_ips_ida:.3f}).')
print(f'Isso sugere que aspectos psicossociais, isoladamente, não são fortes preditores de queda de desempenho.')
print(f'Porém, alunos no quartil mais baixo de IPS têm INDE médio de {ips_quartil_stats.loc["Q1 (Baixo)", "INDE"]:.2f},')
print(f'enquanto os do quartil mais alto têm INDE médio de {ips_quartil_stats.loc["Q4 (Alto)", "INDE"]:.2f}.')

---

## 📋 PERGUNTA 6: Aspectos Psicopedagógicos (IPP)

**As avaliações psicopedagógicas (IPP) confirmam ou contradizem a defasagem identificada pelo IAN?**

In [ ]:
print('='*70)
print('PERGUNTA 6: RELAÇÃO ENTRE IPP E DEFASAGEM (IAN)')
print('='*70)

# Verificar se IPP existe nos dados
if 'IPP' in df_unificado.columns:
    # Correlação IPP x IAN
    corr_ipp_ian = df_unificado[['IPP', 'IAN']].corr().iloc[0, 1]
    corr_ipp_defasagem = df_unificado[['IPP', 'DEFASAGEM']].corr().iloc[0, 1]
    
    print(f'\n📊 Correlações do IPP (Psicopedagógico):')
    print(f'   IPP x IAN: {corr_ipp_ian:.3f}')
    print(f'   IPP x DEFASAGEM: {corr_ipp_defasagem:.3f}')
    
    # Análise por categoria de defasagem
    ipp_por_defasagem = df_unificado.groupby('CATEGORIA_DEFASAGEM')['IPP'].mean()
    print(f'\n📈 IPP médio por categoria de defasagem:')
    for cat, val in ipp_por_defasagem.items():
        print(f'   {cat}: {val:.2f}')
else:
    print('\n⚠️ Coluna IPP não disponível em todos os anos.')
    print('Análise limitada aos anos com dados disponíveis.')

In [ ]:
# Visualização
if 'IPP' in df_unificado.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Gráfico 1: IPP vs IAN
    axes[0].scatter(df_unificado['IPP'], df_unificado['IAN'], alpha=0.3, c='#2ecc71')
    axes[0].set_title(f'IPP vs IAN (r={corr_ipp_ian:.3f})', fontweight='bold')
    axes[0].set_xlabel('IPP (Psicopedagógico)')
    axes[0].set_ylabel('IAN (Adequação ao Nível)')
    
    # Gráfico 2: IPP por categoria de defasagem
    ordem_cat = ['Severamente defasado (≤-3)', 'Moderadamente defasado (-2 a -1)', 
                 'Adequado ao nível (0)', 'Acima do nível (≥1)']
    ipp_cat = df_unificado.groupby('CATEGORIA_DEFASAGEM')['IPP'].mean().reindex([c for c in ordem_cat if c in df_unificado['CATEGORIA_DEFASAGEM'].unique()])
    cores_cat = ['#d62728', '#ff7f0e', '#2ca02c', '#1f77b4']
    bars = axes[1].bar(range(len(ipp_cat)), ipp_cat.values, color=cores_cat[:len(ipp_cat)])
    axes[1].set_title('IPP Médio por Categoria de Defasagem', fontweight='bold')
    axes[1].set_xlabel('Categoria de Defasagem')
    axes[1].set_ylabel('IPP Médio')
    axes[1].set_xticks(range(len(ipp_cat)))
    axes[1].set_xticklabels([c.split('(')[0].strip() for c in ipp_cat.index], rotation=15, ha='right')
    
    plt.tight_layout()
    plt.savefig('pergunta6_psicopedagogico.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f'\n📊 CONCLUSÃO PERGUNTA 6:')
    if abs(corr_ipp_ian) > 0.3:
        print(f'As avaliações psicopedagógicas (IPP) CONFIRMAM parcialmente a defasagem (IAN).')
    else:
        print(f'As avaliações psicopedagógicas (IPP) têm correlação fraca com a defasagem (IAN).')
    print(f'Correlação IPP x IAN: {corr_ipp_ian:.3f}')

---

## 📋 PERGUNTA 7: Ponto de Virada (IPV)

**Quais comportamentos - acadêmicos, emocionais ou de engajamento - mais influenciam o IPV ao longo do tempo?**

In [ ]:
print('='*70)
print('PERGUNTA 7: FATORES QUE INFLUENCIAM O PONTO DE VIRADA (IPV)')
print('='*70)

# Correlações com IPV
indicadores = ['IDA', 'IEG', 'IAA', 'IPS', 'IPP', 'IAN']
correlacoes_ipv = {}

for ind in indicadores:
    if ind in df_unificado.columns:
        corr = df_unificado[['IPV', ind]].corr().iloc[0, 1]
        correlacoes_ipv[ind] = corr

# Ordenar por correlação
correlacoes_ipv = dict(sorted(correlacoes_ipv.items(), key=lambda x: abs(x[1]), reverse=True))

print(f'\n📊 Correlações com IPV (Ponto de Virada):')
for ind, corr in correlacoes_ipv.items():
    interpretacao = interpretar_correlacao(corr)
    print(f'   {ind}: {corr:.3f} ({interpretacao})')

In [ ]:
# Visualização
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Correlações com IPV
inds = list(correlacoes_ipv.keys())
corrs = list(correlacoes_ipv.values())
cores = ['#2ecc71' if c > 0 else '#e74c3c' for c in corrs]
bars = axes[0].barh(inds, corrs, color=cores, edgecolor='black')
axes[0].axvline(x=0, color='black', linewidth=0.5)
axes[0].set_title('Correlação dos Indicadores com IPV', fontweight='bold')
axes[0].set_xlabel('Correlação')
for bar, val in zip(bars, corrs):
    axes[0].annotate(f'{val:.3f}', (val, bar.get_y() + bar.get_height()/2), 
                     textcoords='offset points', xytext=(5 if val > 0 else -35, 0), va='center', fontweight='bold')

# Gráfico 2: Top 2 indicadores vs IPV
top_ind = list(correlacoes_ipv.keys())[:2]
for i, ind in enumerate(top_ind):
    axes[1].scatter(df_unificado[ind], df_unificado['IPV'], alpha=0.3, label=ind)

axes[1].set_title(f'IPV vs Principais Indicadores', fontweight='bold')
axes[1].set_xlabel('Valor do Indicador')
axes[1].set_ylabel('IPV (Ponto de Virada)')
axes[1].legend()

plt.tight_layout()
plt.savefig('pergunta7_ponto_virada.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n📊 CONCLUSÃO PERGUNTA 7:')
print(f'Os indicadores que MAIS influenciam o Ponto de Virada (IPV) são:')
for i, (ind, corr) in enumerate(list(correlacoes_ipv.items())[:3], 1):
    print(f'   {i}. {ind}: r = {corr:.3f}')
print(f'\nO engajamento (IEG) e o desempenho acadêmico (IDA) são os principais preditores do ponto de virada.')

---

## 📋 PERGUNTA 8: Multidimensionalidade dos Indicadores

**Quais combinações de indicadores (IDA + IEG + IPS + IPP) melhor explicam o desempenho global do aluno (INDE)?**

In [ ]:
print('='*70)
print('PERGUNTA 8: COMBINAÇÃO DE INDICADORES QUE EXPLICAM O INDE')
print('='*70)

# Correlações com INDE
indicadores = ['IDA', 'IEG', 'IAA', 'IPS', 'IPP', 'IPV', 'IAN']
correlacoes_inde = {}

for ind in indicadores:
    if ind in df_unificado.columns:
        corr = df_unificado[['INDE', ind]].corr().iloc[0, 1]
        correlacoes_inde[ind] = corr

correlacoes_inde = dict(sorted(correlacoes_inde.items(), key=lambda x: abs(x[1]), reverse=True))

print(f'\n📊 Correlações com INDE (Índice Global):')
for ind, corr in correlacoes_inde.items():
    print(f'   {ind}: {corr:.3f} ({interpretar_correlacao(corr)})')

# Regressão múltipla simplificada
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Preparar dados
features = ['IDA', 'IEG', 'IAA', 'IPS', 'IPV', 'IAN']
features_disp = [f for f in features if f in df_unificado.columns]

df_reg = df_unificado[features_disp + ['INDE']].dropna()

X = df_reg[features_disp]
y = df_reg['INDE']

# Normalizar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Regressão
reg = LinearRegression()
reg.fit(X_scaled, y)

print(f'\n📈 Importância dos indicadores (coeficientes padronizados):')
coefs = dict(zip(features_disp, reg.coef_))
coefs = dict(sorted(coefs.items(), key=lambda x: abs(x[1]), reverse=True))
for ind, coef in coefs.items():
    print(f'   {ind}: {coef:.3f}')

print(f'\nR² do modelo: {reg.score(X_scaled, y):.3f}')

In [ ]:
# Visualização
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Correlações com INDE
inds = list(correlacoes_inde.keys())
corrs = list(correlacoes_inde.values())
cores = ['#3498db' if c > 0.5 else '#95a5a6' for c in corrs]
bars = axes[0].barh(inds, corrs, color=cores, edgecolor='black')
axes[0].set_title('Correlação dos Indicadores com INDE', fontweight='bold')
axes[0].set_xlabel('Correlação')
axes[0].set_xlim(0, 1)
for bar, val in zip(bars, corrs):
    axes[0].annotate(f'{val:.3f}', (val, bar.get_y() + bar.get_height()/2), 
                     textcoords='offset points', xytext=(5, 0), va='center', fontweight='bold')

# Gráfico 2: Coeficientes da regressão
inds_reg = list(coefs.keys())
coefs_vals = list(coefs.values())
cores_reg = ['#2ecc71' if c > 0 else '#e74c3c' for c in coefs_vals]
bars = axes[1].barh(inds_reg, coefs_vals, color=cores_reg, edgecolor='black')
axes[1].axvline(x=0, color='black', linewidth=0.5)
axes[1].set_title('Importância dos Indicadores (Regressão)', fontweight='bold')
axes[1].set_xlabel('Coeficiente Padronizado')
for bar, val in zip(bars, coefs_vals):
    axes[1].annotate(f'{val:.3f}', (val, bar.get_y() + bar.get_height()/2), 
                     textcoords='offset points', xytext=(5 if val > 0 else -35, 0), va='center', fontweight='bold')

plt.tight_layout()
plt.savefig('pergunta8_multidimensional.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n📊 CONCLUSÃO PERGUNTA 8:')
print(f'As combinações de indicadores que MELHOR explicam o INDE são:')
top_3 = list(coefs.items())[:3]
for i, (ind, coef) in enumerate(top_3, 1):
    print(f'   {i}. {ind} (coef={coef:.3f})')
print(f'\nO modelo com todos os indicadores explica {reg.score(X_scaled, y)*100:.1f}% da variação do INDE.')

---

## 📋 PERGUNTA 9: Previsão de Risco com Machine Learning

**Quais padrões nos indicadores permitem identificar alunos em risco antes de queda no desempenho ou aumento da defasagem?**

*Esta pergunta será respondida em detalhes no Notebook 3 (Modelo Preditivo).*

In [ ]:
print('='*70)
print('PERGUNTA 9: IDENTIFICAÇÃO DE ALUNOS EM RISCO')
print('='*70)

# Definir alunos em risco (defasagem severa ou moderada)
df_unificado['EM_RISCO'] = df_unificado['DEFASAGEM'].apply(lambda x: 1 if x <= -2 else 0)

# Estatísticas
total_risco = df_unificado['EM_RISCO'].sum()
pct_risco = total_risco / len(df_unificado) * 100

print(f'\n📊 Alunos em Risco (defasagem ≤ -2 anos):')
print(f'   Total: {total_risco:,} alunos ({pct_risco:.1f}%)')

# Perfil dos alunos em risco
print(f'\n📈 Perfil médio dos alunos em risco vs não em risco:')
perfil = df_unificado.groupby('EM_RISCO')[['INDE', 'IDA', 'IEG', 'IAA', 'IPS', 'IPV', 'IAN']].mean()
perfil.index = ['Não em risco', 'Em risco']
print(perfil.round(2))

print(f'\n⚠️ Análise detalhada e modelo preditivo serão desenvolvidos no Notebook 3.')

---

## 📋 PERGUNTA 10: Efetividade do Programa

**Os indicadores mostram melhora consistente ao longo do ciclo nas diferentes fases (Quartzo, Ágata, Ametista e Topázio), confirmando o impacto real do programa?**

In [ ]:
print('='*70)
print('PERGUNTA 10: EFETIVIDADE DO PROGRAMA PASSOS MÁGICOS')
print('='*70)

# Análise por Pedra
ordem_pedras = ['Quartzo', 'Ágata', 'Ametista', 'Topázio']
indicadores = ['INDE', 'IDA', 'IEG', 'IAA', 'IPS', 'IPV', 'IAN']

print(f'\n📊 Médias dos Indicadores por Pedra (classificação INDE):')
medias_pedra = df_unificado.groupby('PEDRA')[indicadores].mean()
medias_pedra = medias_pedra.reindex([p for p in ordem_pedras if p in medias_pedra.index])
print(medias_pedra.round(2))

# Evolução da distribuição de pedras
print(f'\n📈 Evolução da distribuição de Pedras por ano:')
pedras_ano = df_unificado.groupby(['ANO', 'PEDRA']).size().unstack(fill_value=0)
pedras_ano_pct = pedras_ano.div(pedras_ano.sum(axis=1), axis=0) * 100
pedras_ano_pct = pedras_ano_pct[[p for p in ordem_pedras if p in pedras_ano_pct.columns]]
print(pedras_ano_pct.round(1))

In [ ]:
# Visualização
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Gráfico 1: Evolução das Pedras
cores = [CORES_PEDRAS.get(p, '#808080') for p in pedras_ano_pct.columns]
pedras_ano_pct.plot(kind='bar', stacked=True, ax=axes[0], color=cores, edgecolor='black', linewidth=0.5)
axes[0].set_title('Evolução da Distribuição de Pedras', fontweight='bold')
axes[0].set_xlabel('Ano')
axes[0].set_ylabel('Percentual (%)')
axes[0].legend(title='Pedra', bbox_to_anchor=(1.02, 1))
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)

# Gráfico 2: INDE médio por ano
media_inde = df_unificado.groupby('ANO')['INDE'].mean()
axes[1].plot(media_inde.index, media_inde.values, marker='o', linewidth=3, markersize=12, color='#3498db')
axes[1].fill_between(media_inde.index, media_inde.values, alpha=0.3, color='#3498db')
axes[1].set_title('Evolução do INDE Médio', fontweight='bold')
axes[1].set_xlabel('Ano')
axes[1].set_ylabel('INDE Médio')
for x, y in zip(media_inde.index, media_inde.values):
    axes[1].annotate(f'{y:.2f}', (x, y), textcoords='offset points', xytext=(0, 10), ha='center', fontweight='bold')

# Gráfico 3: Percentual de Topázio + Ametista por ano
pct_alto = pedras_ano_pct[['Ametista', 'Topázio']].sum(axis=1)
axes[2].bar(pct_alto.index, pct_alto.values, color='#8B008B', edgecolor='black')
axes[2].set_title('% Alunos em Ametista + Topázio', fontweight='bold')
axes[2].set_xlabel('Ano')
axes[2].set_ylabel('Percentual (%)')
for i, (x, y) in enumerate(zip(pct_alto.index, pct_alto.values)):
    axes[2].annotate(f'{y:.1f}%', (x, y), textcoords='offset points', xytext=(0, 5), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('pergunta10_efetividade.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n📊 CONCLUSÃO PERGUNTA 10:')
print(f'O programa mostra IMPACTO POSITIVO ao longo dos anos:')
print(f'  • INDE médio: {media_inde[2022]:.2f} (2022) → {media_inde[2024]:.2f} (2024)')
print(f'  • % em Ametista+Topázio: {pct_alto[2022]:.1f}% (2022) → {pct_alto[2024]:.1f}% (2024)')
print(f'  • Defasagem média: {df_unificado[df_unificado["ANO"]==2022]["DEFASAGEM"].mean():.2f} (2022) → {df_unificado[df_unificado["ANO"]==2024]["DEFASAGEM"].mean():.2f} (2024)')
print(f'\nOs dados confirmam a efetividade do programa Passos Mágicos!')

---

## 📋 PERGUNTA 11: Insights e Criatividade

**Adicionar mais insights e pontos de vista não abordados nas perguntas, utilizando criatividade e análise dos dados para trazer sugestões para a Passos Mágicos.**

In [ ]:
print('='*70)
print('PERGUNTA 11: INSIGHTS ADICIONAIS E SUGESTÕES')
print('='*70)

# Insight 1: Análise por gênero
print(f'\n📊 INSIGHT 1: Análise por Gênero')
if 'GENERO' in df_unificado.columns:
    genero_stats = df_unificado.groupby('GENERO')[['INDE', 'IDA', 'IEG']].mean()
    print(genero_stats.round(2))

# Insight 2: Alunos que mais evoluíram
print(f'\n📊 INSIGHT 2: Perfil dos alunos Topázio (melhor desempenho)')
topazio = df_unificado[df_unificado['PEDRA'] == 'Topázio']
print(f'   Total de alunos Topázio: {len(topazio):,}')
print(f'   INDE médio: {topazio["INDE"].mean():.2f}')
print(f'   IDA médio: {topazio["IDA"].mean():.2f}')
print(f'   IEG médio: {topazio["IEG"].mean():.2f}')

# Insight 3: Alunos em risco
print(f'\n📊 INSIGHT 3: Perfil dos alunos Quartzo (maior risco)')
quartzo = df_unificado[df_unificado['PEDRA'] == 'Quartzo']
print(f'   Total de alunos Quartzo: {len(quartzo):,}')
print(f'   INDE médio: {quartzo["INDE"].mean():.2f}')
print(f'   IDA médio: {quartzo["IDA"].mean():.2f}')
print(f'   IEG médio: {quartzo["IEG"].mean():.2f}')

In [ ]:
# Sugestões baseadas nos dados
print('\n' + '='*70)
print('💡 SUGESTÕES PARA A PASSOS MÁGICOS')
print('='*70)

print('''
1. FOCO NO ENGAJAMENTO (IEG)
   O engajamento é o indicador com maior correlação com o INDE e IPV.
   Sugestão: Desenvolver programas específicos para aumentar o engajamento
   dos alunos, especialmente aqueles classificados como Quartzo.

2. INTERVENÇÃO PRECOCE
   Alunos com defasagem ≤ -2 anos representam um grupo de risco.
   Sugestão: Implementar sistema de alerta precoce baseado nos indicadores
   para identificar alunos em risco antes da queda de desempenho.

3. ACOMPANHAMENTO PSICOSSOCIAL
   Embora o IPS tenha correlação fraca com desempenho, alunos no quartil
   mais baixo de IPS têm INDE significativamente menor.
   Sugestão: Fortalecer o acompanhamento psicossocial para alunos vulneráveis.

4. AUTOAVALIAÇÃO COMO FERRAMENTA
   A discrepância entre IAA e IDA pode indicar necessidade de orientação.
   Sugestão: Usar a autoavaliação como ferramenta de desenvolvimento
   de metacognição e autoconhecimento dos alunos.

5. CELEBRAR O SUCESSO
   O aumento de alunos em Ametista e Topázio demonstra o impacto positivo.
   Sugestão: Criar programa de mentoria onde alunos Topázio apoiem
   alunos Quartzo, fortalecendo a comunidade.
''')

---

## 📝 Resumo das Conclusões

| Pergunta | Conclusão Principal |
|----------|--------------------|
| 1. IAN (Defasagem) | Defasagem média melhorou de -0.94 (2022) para -0.41 (2024) |
| 2. IDA (Desempenho) | Desempenho acadêmico relativamente estável ao longo dos anos |
| 3. IEG (Engajamento) | Forte correlação com IDA (r=0.54) e IPV (r=0.56) |
| 4. IAA (Autoavaliação) | Parcialmente coerente com desempenho real |
| 5. IPS (Psicossocial) | Correlação fraca com desempenho, mas impacto em casos extremos |
| 6. IPP (Psicopedagógico) | Confirma parcialmente a defasagem identificada |
| 7. IPV (Ponto de Virada) | IEG e IDA são os principais preditores |
| 8. Multidimensionalidade | IDA, IEG e IPV explicam maior parte do INDE |
| 9. Previsão de Risco | Modelo preditivo desenvolvido no Notebook 3 |
| 10. Efetividade | Programa mostra impacto positivo consistente |
| 11. Insights | Sugestões para intervenção precoce e foco em engajamento |

---